In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark import SparkContext, SparkConf, SQLContext

from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import pandas_udf
from pyspark.sql.window import Window

import pandas as pd
import numpy as np

from datetime import date, datetime, timedelta, timezone
import os
import json
import requests
import time
import yaml
from itertools import chain

import concurrent.futures
from threading import Lock

In [2]:
appName = "PySpark TFT puuids"
master = "local[*]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.executor.memory", "40g") \
    .set("spark.driver.memory", "40g") \
    .set("spark.executor.memoryOverhead", "8g") \
    .set("spark.local.dir", "/home/mai/spark-temp") \
    .set("spark.sql.session.timeZone", "UTC") \
    .set("spark.dynamicAllocation.enabled", "true") \
    .set("spark.dynamicAllocation.minExecutors", "2") \
    .set("spark.dynamicAllocation.maxExecutors", "50") \
    .set("spark.speculation", "true") 
   
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

25/02/19 22:48:46 WARN Utils: Your hostname, LAPTOP-4O0SI9BK resolves to a loopback address: 127.0.1.1; using 172.30.79.188 instead (on interface eth0)
25/02/19 22:48:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/19 22:48:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/19 22:48:48 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Set up

In [3]:
with open('./api_key.yaml', 'r') as file:
    API_KEY = yaml.safe_load(file)['api_key']

## Get match data

In [4]:
REGION = 'euw1'
REGION_TO_EXECUTE = 'EUROPE'
BASE_URL = f'https://{REGION_TO_EXECUTE}.api.riotgames.com/tft/match/v1/matches'
TIER = 'GOLD'
DIVISION = 'I'

In [6]:
# get the match id
# df = spark.read.parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}.parquet')
# exclude = spark.read.parquet(f'./data/tft_matches_id/match_id_{REGION}_CHALLENGER_{DIVISION}.parquet')

In [7]:
# df.show(5)

In [8]:
# df.count()

In [9]:
# # filter out the match ids already existing in challenger ranked players match history
# df = df.join(exclude, on=['match_id'], how='leftanti')

In [10]:
# match_ids = [row['match_id'] for row in df.select('match_id').collect()]

In [11]:
# len(match_ids)

In [12]:
# match_ids_1 = match_ids[:8000]
# match_ids_2 = match_ids[8000:]

In [13]:
# # save into 2 parts
# pd.DataFrame(match_ids_1, columns=['match_id']).to_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p1.parquet', index=False)
# pd.DataFrame(match_ids_2, columns=['match_id']).to_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p2.parquet', index=False)

In [5]:
match_ids_2 = pd.read_parquet(f'./data/tft_matches_id/match_id_{REGION}_{TIER}_{DIVISION}_p2.parquet')['match_id'].to_list()

In [7]:
len(match_ids_2)

7539

In [8]:
# Limits
MAX_WORKERS = 20
REQS_PER_SEC = 20
REQS_PER_2MIN = 100

In [9]:
# global counters
request_counter = 0
lock = Lock()          # ensure thread safety for counter updates

In [10]:
# Create a global session
session = requests.Session()
session.headers.update({"X-Riot-Token": API_KEY,
                       'Connection': 'keep-alive'})

In [11]:
def get_match_data(match_id, max_retries=5):
    global request_counter
    # construct url
    url = f'{BASE_URL}/{match_id}'

    for attempt in range(max_retries):
        try:
            response = session.get(url, timeout=10)
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get("Retry-After", 5))
                # print(f"Rate limit exceeded. Waiting {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"Error {response.status_code}: {response.text}")
                return None
        except requests.exceptions.ConnectionError as e:
            # print(f"Connection error: {e}. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)  # Exponential backoff

        # response = session.get(url)
        # if response.status_code == 200:
        #     # Update request counter safely
        #     with lock:
        #         request_counter += 1
        #     return response.json()
        # elif response.status_code == 429:   # rate limit hit
        #     retry_after = int(response.headers.get('Retry-After', 2))
        #     # print(f"Rate limit exceeded. Waiting {retry_after} seconds...")
        #     time.sleep(retry_after)
        # else:
        #     print(f"Error {response.status_code} for match {match_id}: {response.text}")
        #     return None  # Skip failed requests

In [13]:
match_data = get_match_data(match_ids_2[0])

match_data

{'metadata': {'data_version': '6',
  'match_id': 'EUW1_7277970420',
  'participants': ['qCf510udus60bm7ugdNGnunhZF0HgdlX50ZeFaKzs9igpR_q78R50soPh9NDOxIfbZs1H3tcBKngeg',
   'l1NGMDT4smt0g5RZL_NkHSuo3c4w6_1quWlnez3uzLarPCVyG2aU1TY-WPSthWZp6CrLQG7RPk71lw',
   'GvHVTT-RTIOECvqWirlTU1oawN-Cs5VP8Qx01nClHJP9kr0jLLo3kjUWh-RYcHZ2vocpfZMY82z76Q',
   'EozPqw1VzjOiZC3YAlGS9Eh9uR3BpnSEKaUR-qJIVVO-fMSDALxBFjG9XTOlDY-8lTZlBMb6ImN_5g',
   'mURlCDRJWHRugHqkUraQPFdTrW7wO84HsLE7LSvio0s67rNXD9Liqp_v04c4EIE4e2rSV9JcwI3EbQ',
   '9sY-dnE744eF0_LjDGwRllqUMUa0C5AGGyD4d282mvW1RVReMg7060D2ugl92X5gDkqyOIoDdTDx_A',
   '8i2Qm-yO8RzKzvXW3LTyhEVLf7guaKA2FQxkE3vVLEEdjIgbc77TSmdAFLtCCEl4ek_HG1dTTW3rDQ',
   'NnGxf8XqwGzInJHqTFXiv139JSlYGD1hXyTXPvoyxln8YtudtpbiD9-YiNJ_dFmK3_u9tG871znQUg']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1737595671000,
  'gameId': 7277970420,
  'game_datetime': 1737597885965,
  'game_length': 2196.9853515625,
  'game_version': 'Linux Version 15.1.649.4112 (Jan 13 2025/15:4

In [15]:
with open('./match_data_test.json', 'w', encoding='utf-8') as file:
    json.dump(match_data, file, indent=4)

In [20]:
def get_match_data_parallel(match_ids_):
    global request_counter
    match_data = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = []
        for i, match_id in enumerate(match_ids_):
            futures.append(executor.submit(get_match_data, match_id))  # Correct function call
            
            # Rate limit handling
            if (i + 1) % REQS_PER_SEC == 0:
                print(f"Reached {REQS_PER_SEC} requests, sleeping 1 second...")
                time.sleep(1)
            if (i + 1) % REQS_PER_2MIN == 0:
                print(f"Reached {REQS_PER_2MIN} requests, sleeping 120 seconds...")
                time.sleep(120)

        results = [future.result() for future in futures]  # Get results
    return [data for data in results if data is not None]  # Filter out failed requests

In [ ]:
%%time
match_data = get_match_data_parallel(match_ids_2[3750:])

Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 100 requests, sleeping 120 seconds...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 100 requests, sleeping 120 seconds...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 100 requests, sleeping 120 seconds...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 20 requests, sleeping 1 second...
Reached 100 requests, 

In [ ]:
len(match_data)

In [ ]:
df = pd.json_normalize(match_data).explode('info.participants')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
# df[df['metadata.match_id'] == 'KR_7507653169']

In [ ]:
df[df['info.queueId'] == 1100].shape

In [ ]:
df['info.participants'].head()

In [ ]:
# convert to parquet
match_df = spark.createDataFrame(df)

In [ ]:
match_df.write.mode('overwrite').parquet(f'./data/tft_match_data/matches_{REGION}_{TIER}_{DIVISION}_p3.parquet')

In [ ]:
df.to_parquet(f'./data/tft_match_data_pandas/matches_{REGION}_{TIER}_{DIVISION}_p3.parquet')